# Создается датасет - характеризующий параметры автомобиля:
- max_sp: максимальная скорость, км/ч
- ac_one: разгон до 100 км/ч, сек
- v: объем двигателя, см3
- hp: количество л/с ( > 200 - 1, < 200 - 0)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from sklearn.linear_model import LinearRegression
import pickle

# data_creation.py

# создание датасета
max_sp = np.random.normal(180, 40, 5000) + np.random.uniform(0, 10, 5000)
ac_one = np.random.normal(8, 6, 5000) + np.random.uniform(0, 2, 5000)
v = np.random.normal(2000, 1000, 5000)

data = {'max_sp': max_sp, 'ac_one': ac_one, 'v': v} 
df = pd.DataFrame(data) 

# если разгон до 100км/ч меньше 6 секунд - кол-во л/с больше 200
df['hp'] = df['ac_one'].apply(lambda x: 1 if x < 6 else 0)

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Создание train и test
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('test'):
    os.makedirs('test')
    

df_train.to_csv('train/df_train.csv', index=False)
df_test.to_csv('test/df_test.csv', index=False)



# ----------------

df_train = pd.read_csv('train/df_train.csv')
df_test = pd.read_csv('test/df_test.csv')

# принимаем, что разгон до 100 км/ч менее 3 сек - выбросы
filt_train = ((df_train['ac_one'] > 3))
filt_test = ((df_test['ac_one'] > 3))
df_train, df_test = df_train[filt_train], df_test[filt_test]

# Стандартизация
scaler = StandardScaler()
df_train.iloc[:, :-1] = scaler.fit_transform(df_train.iloc[:, :-1])
df_test.iloc[:, :-1] = scaler.fit_transform(df_test.iloc[:, :-1])

# Сохранение обработанных данных в файлы
if not os.path.exists('preprocessed_train'):
    os.makedirs('preprocessed_train')
if not os.path.exists('preprocessed_test'):
    os.makedirs('preprocessed_test')

df_train.to_csv('train/preprocessed_train.csv', index=False)
df_test.to_csv('test/preprocessed_test.csv', index=False)


# model_preparation.py

df_train = pd.read_csv('train/preprocessed_train.csv')

X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values

model = LinearRegression(fit_intercept=False)
reg = model.fit(X_train, y_train)

pkl_filename = "reg_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(model, file) 
    
    
# model_testing.py

pkl_filename = "reg_model.pkl" 
with open(pkl_filename, 'rb') as file: 
    model = pickle.load(file) 

df_test = pd.read_csv('test/preprocessed_test.csv')
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

print(f'Model test accuracy is: {round(model.score(X_test, y_test),3)}')

Model test accuracy is: 0.146
